In [1]:
import numpy as np
import os, json, cv2, random
import skimage.io as io
import pylab
import sys
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
from tensorboard.backend.event_processing import event_accumulator as ea
from PIL import Image
import shutil

In [2]:
# Category file, one category per line
yolo_format_classes_path = 'dataset/classes.txt' #absolute path to the file where the categories are stored. The file should contain one category per line.
# Write the category according to your own data set. 

#Read the categories file and extract all categories
with open(yolo_format_classes_path,'r') as f1:
    lines1 = f1.readlines()
categories = []
for j,label in enumerate(lines1):
    label = label.strip()
    categories.append({'id':j+1,'name':label,'supercategory': label})
    
write_json_context = dict()
write_json_context['info'] = {'description': '', 'url': '', 'version': '', 'year': 2021, 'contributor': '', 'date_created': '2021-02-12 11:00:08.5'}
write_json_context['licenses'] = [{'id': 1, 'name': None, 'url': None}]
write_json_context['categories'] = categories
write_json_context['images'] = []
write_json_context['annotations'] = []

In [8]:
import os
import json
import cv2

# Initialize the context for the COCO format
write_json_context = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "WBC", "supercategory": "WBC"},
        {"id": 2, "name": "RBC", "supercategory": "RBC"},
        {"id": 3, "name": "Platelet", "supercategory": "Platelet"}
    ],
    "info": {
        "description": "Dataset for classification",
        "url": "",
        "version": "1.0",
        "year": 2021,
        "contributor": "",
        "date_created": "2021-02-12 11:00:08.5"
    },
    "licenses": [{"id": 1, "name": "Unknown", "url": ""}]
}

# Initialize the image and annotation counters
file_number = 1
num_bboxes = 1  # Global annotation ID counter, maintained across all runs

# Directories for images and annotations
directory_labels = os.fsencode("dataset/labels/train")
directory_images = os.fsencode("dataset/images/train")

# Loop through images in the 'train' folder
for file in os.listdir(directory_images):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"):
        img_path = os.path.join(directory_images.decode("utf-8"), filename)
        base = os.path.basename(img_path)
        file_name_without_ext = os.path.splitext(base)[0]  # name of the file without extension
        yolo_annotation_path = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext + "." + 'txt')

        # Image context info
        img_name = os.path.basename(img_path)
        img_context = {}
        height, width = cv2.imread(img_path).shape[:2]
        img_context['file_name'] = img_name
        img_context['height'] = height
        img_context['width'] = width
        img_context['date_captured'] = '2021-02-12 11:00:08.5'
        img_context['id'] = file_number  # image id
        img_context['license'] = 1
        img_context['coco_url'] = ''
        img_context['flickr_url'] = ''
        write_json_context['images'].append(img_context)

        # Read YOLO annotations and convert to COCO format
        with open(yolo_annotation_path, 'r') as f2:
            lines2 = f2.readlines()

        for i, line in enumerate(lines2):  # Loop through each annotation in the image
            line = line.split(' ')
            bbox_dict = {}
            class_id, x_yolo, y_yolo, width_yolo, height_yolo = line[0:]
            x_yolo, y_yolo, width_yolo, height_yolo, class_id = float(x_yolo), float(y_yolo), float(width_yolo), float(height_yolo), int(class_id)
            bbox_dict['id'] = num_bboxes  # Assign unique annotation ID across all files
            bbox_dict['image_id'] = file_number
            bbox_dict['category_id'] = class_id + 1  # COCO categories start at 1, YOLO starts at 0
            bbox_dict['iscrowd'] = 0  # Not marked as crowd

            # Calculate the absolute dimensions of the bounding box
            h, w = abs(height_yolo * height), abs(width_yolo * width)
            bbox_dict['area'] = h * w
            x_coco = round(x_yolo * width - (w / 2))
            y_coco = round(y_yolo * height - (h / 2))

            # Ensure bounding box coordinates are within the image
            if x_coco < 0:
                x_coco = 1
            if y_coco < 0:
                y_coco = 1

            bbox_dict['bbox'] = [x_coco, y_coco, w, h]
            bbox_dict['segmentation'] = [[x_coco, y_coco, x_coco + w, y_coco, x_coco + w, y_coco + h, x_coco, y_coco + h]]

            write_json_context['annotations'].append(bbox_dict)
            num_bboxes += 1  # Increment the annotation ID

        file_number += 1  # Increment image ID
        continue
    else:
        continue

# Finally, save the result to a JSON file
coco_format_save_path = 'dataset/images/train/train.json'
with open(coco_format_save_path, 'w') as fw:
    json.dump(write_json_context, fw)

print(f"COCO formatted JSON saved to {coco_format_save_path}")


COCO formatted JSON saved to dataset/images/train/train.json


In [9]:
import os
import json
import cv2

# Initialize the context for the COCO format
write_json_context = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "WBC", "supercategory": "WBC"},
        {"id": 2, "name": "RBC", "supercategory": "RBC"},
        {"id": 3, "name": "Platelet", "supercategory": "Platelet"}
    ],
    "info": {
        "description": "Dataset for classification",
        "url": "",
        "version": "1.0",
        "year": 2021,
        "contributor": "",
        "date_created": "2021-02-12 11:00:08.5"
    },
    "licenses": [{"id": 1, "name": "Unknown", "url": ""}]
}

# Initialize the image and annotation counters
file_number = 1
num_bboxes = 1  # Global annotation ID counter, maintained across all runs

# Directories for images and annotations
directory_labels = os.fsencode("dataset/labels/test")
directory_images = os.fsencode("dataset/images/test")

# Loop through images in the 'train' folder
for file in os.listdir(directory_images):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"):
        img_path = os.path.join(directory_images.decode("utf-8"), filename)
        base = os.path.basename(img_path)
        file_name_without_ext = os.path.splitext(base)[0]  # name of the file without extension
        yolo_annotation_path = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext + "." + 'txt')

        # Image context info
        img_name = os.path.basename(img_path)
        img_context = {}
        height, width = cv2.imread(img_path).shape[:2]
        img_context['file_name'] = img_name
        img_context['height'] = height
        img_context['width'] = width
        img_context['date_captured'] = '2021-02-12 11:00:08.5'
        img_context['id'] = file_number  # image id
        img_context['license'] = 1
        img_context['coco_url'] = ''
        img_context['flickr_url'] = ''
        write_json_context['images'].append(img_context)

        # Read YOLO annotations and convert to COCO format
        with open(yolo_annotation_path, 'r') as f2:
            lines2 = f2.readlines()

        for i, line in enumerate(lines2):  # Loop through each annotation in the image
            line = line.split(' ')
            bbox_dict = {}
            class_id, x_yolo, y_yolo, width_yolo, height_yolo = line[0:]
            x_yolo, y_yolo, width_yolo, height_yolo, class_id = float(x_yolo), float(y_yolo), float(width_yolo), float(height_yolo), int(class_id)
            bbox_dict['id'] = num_bboxes  # Assign unique annotation ID across all files
            bbox_dict['image_id'] = file_number
            bbox_dict['category_id'] = class_id + 1  # COCO categories start at 1, YOLO starts at 0
            bbox_dict['iscrowd'] = 0  # Not marked as crowd

            # Calculate the absolute dimensions of the bounding box
            h, w = abs(height_yolo * height), abs(width_yolo * width)
            bbox_dict['area'] = h * w
            x_coco = round(x_yolo * width - (w / 2))
            y_coco = round(y_yolo * height - (h / 2))

            # Ensure bounding box coordinates are within the image
            if x_coco < 0:
                x_coco = 1
            if y_coco < 0:
                y_coco = 1

            bbox_dict['bbox'] = [x_coco, y_coco, w, h]
            bbox_dict['segmentation'] = [[x_coco, y_coco, x_coco + w, y_coco, x_coco + w, y_coco + h, x_coco, y_coco + h]]

            write_json_context['annotations'].append(bbox_dict)
            num_bboxes += 1  # Increment the annotation ID

        file_number += 1  # Increment image ID
        continue
    else:
        continue

# Finally, save the result to a JSON file
coco_format_save_path = 'dataset/images/test/test.json'
with open(coco_format_save_path, 'w') as fw:
    json.dump(write_json_context, fw)

print(f"COCO formatted JSON saved to {coco_format_save_path}")


COCO formatted JSON saved to dataset/images/test/test.json


In [10]:
import os
import json
import cv2

# Initialize the context for the COCO format
write_json_context = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "WBC", "supercategory": "WBC"},
        {"id": 2, "name": "RBC", "supercategory": "RBC"},
        {"id": 3, "name": "Platelet", "supercategory": "Platelet"}
    ],
    "info": {
        "description": "Dataset for classification",
        "url": "",
        "version": "1.0",
        "year": 2021,
        "contributor": "",
        "date_created": "2021-02-12 11:00:08.5"
    },
    "licenses": [{"id": 1, "name": "Unknown", "url": ""}]
}

# Initialize the image and annotation counters
file_number = 1
num_bboxes = 1  # Global annotation ID counter, maintained across all runs

# Directories for images and annotations
directory_labels = os.fsencode("dataset/labels/val")
directory_images = os.fsencode("dataset/images/val")

# Loop through images in the 'train' folder
for file in os.listdir(directory_images):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"):
        img_path = os.path.join(directory_images.decode("utf-8"), filename)
        base = os.path.basename(img_path)
        file_name_without_ext = os.path.splitext(base)[0]  # name of the file without extension
        yolo_annotation_path = os.path.join(directory_labels.decode("utf-8"), file_name_without_ext + "." + 'txt')

        # Image context info
        img_name = os.path.basename(img_path)
        img_context = {}
        height, width = cv2.imread(img_path).shape[:2]
        img_context['file_name'] = img_name
        img_context['height'] = height
        img_context['width'] = width
        img_context['date_captured'] = '2021-02-12 11:00:08.5'
        img_context['id'] = file_number  # image id
        img_context['license'] = 1
        img_context['coco_url'] = ''
        img_context['flickr_url'] = ''
        write_json_context['images'].append(img_context)

        # Read YOLO annotations and convert to COCO format
        with open(yolo_annotation_path, 'r') as f2:
            lines2 = f2.readlines()

        for i, line in enumerate(lines2):  # Loop through each annotation in the image
            line = line.split(' ')
            bbox_dict = {}
            class_id, x_yolo, y_yolo, width_yolo, height_yolo = line[0:]
            x_yolo, y_yolo, width_yolo, height_yolo, class_id = float(x_yolo), float(y_yolo), float(width_yolo), float(height_yolo), int(class_id)
            bbox_dict['id'] = num_bboxes  # Assign unique annotation ID across all files
            bbox_dict['image_id'] = file_number
            bbox_dict['category_id'] = class_id + 1  # COCO categories start at 1, YOLO starts at 0
            bbox_dict['iscrowd'] = 0  # Not marked as crowd

            # Calculate the absolute dimensions of the bounding box
            h, w = abs(height_yolo * height), abs(width_yolo * width)
            bbox_dict['area'] = h * w
            x_coco = round(x_yolo * width - (w / 2))
            y_coco = round(y_yolo * height - (h / 2))

            # Ensure bounding box coordinates are within the image
            if x_coco < 0:
                x_coco = 1
            if y_coco < 0:
                y_coco = 1

            bbox_dict['bbox'] = [x_coco, y_coco, w, h]
            bbox_dict['segmentation'] = [[x_coco, y_coco, x_coco + w, y_coco, x_coco + w, y_coco + h, x_coco, y_coco + h]]

            write_json_context['annotations'].append(bbox_dict)
            num_bboxes += 1  # Increment the annotation ID

        file_number += 1  # Increment image ID
        continue
    else:
        continue

# Finally, save the result to a JSON file
coco_format_save_path = 'dataset/images/val/val.json'
with open(coco_format_save_path, 'w') as fw:
    json.dump(write_json_context, fw)

print(f"COCO formatted JSON saved to {coco_format_save_path}")


COCO formatted JSON saved to dataset/images/val/val.json


In [11]:
import json
from collections import Counter

# Paths to your COCO annotation JSON files
json_paths = {
    "train": "dataset/images/train/train.json",
    "val": "dataset/images/val/val.json",
    "test": "dataset/images/test/test.json"
}

def check_duplicate_annotation_ids(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    annotation_ids = [ann['id'] for ann in data.get('annotations', [])]
    duplicates = [item for item, count in Counter(annotation_ids).items() if count > 1]
    return duplicates

# Run check
for split, path in json_paths.items():
    duplicates = check_duplicate_annotation_ids(path)
    if duplicates:
        print(f"[{split.upper()}] Duplicate annotation IDs found: {duplicates[:10]}{'...' if len(duplicates) > 10 else ''}")
    else:
        print(f"[{split.upper()}] No duplicate annotation IDs found.")


[TRAIN] No duplicate annotation IDs found.
[VAL] No duplicate annotation IDs found.
[TEST] No duplicate annotation IDs found.
